In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor, AdaBoostRegressor
from sklearn.linear_model import ElasticNet
from sklearn.svm import SVR
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.decomposition import PCA
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error, mean_squared_error
from tensorflow import keras

In [2]:

from time import sleep
from winsound import Beep
from pathlib import Path
import yaml

In [3]:
def write_yaml(file_path: str, data: object):
    """
    Write data to a YAML file.

    Args:
        file_path (str): Path of the YAML file.
        data (object): Data to write.

    Returns:
        bool: True if the file was written successfully, False otherwise.
    """
    try:
        with open(Path(file_path), "w") as f:
            yaml.dump(data, f)
        print("YAML file write: OK")
        return True
    except (FileNotFoundError, IOError, yaml.YAMLError) as err:
        print(f"Failed to write YAML file {file_path}: {err}")
        return False

In [4]:
def bee(r: int = 3, f: int = 2500, d: int = 1000, p: float = 1) -> None:
    """
    Emits a series of sounds using the Beep function from the winsound library.

    Args:
        r (int): Number of sound repetitions (default: 3).
        f (int): Sound frequency in Hz (default: 2500).
        d (int): Sound duration in milliseconds (default: 1000).
        p (float): Pause between each sound in seconds (default: 1).

    Returns:
        None
    """
    
    for n in range(r):
        Beep(f, d)
        sleep(p)

## Declaramos variables y cargamos los datos

In [5]:
parameter = {}
score = {}

In [6]:
heavy_df = pd.read_csv("../data/processed/heavy.csv", index_col="Unnamed: 0")

In [7]:
heavy_df.columns

Index(['score', 'read', 'pending', 'following', 'favorite', 'have',
       'abandoned', 'chapters_count', 'Magia_score', 'Mecha_score',
       'Demonios_score', 'Género Bender_score', 'Realidad Virtual_score',
       'Drama_score', 'Niños_score', 'Guerra_score', 'Harem_score',
       'Vampiros_score', 'Horror_score', 'Acción_score', 'Realidad_score',
       'Traps_score', 'Militar_score', 'Crimen_score',
       'Recuentos de la vida_score', 'Apocalíptico_score', 'Psicológico_score',
       'Misterio_score', 'Musica_score', 'Extranjero_score', 'Samurái_score',
       'Girls Love_score', 'Telenovela_score', 'Policiaco_score',
       'Animación_score', 'Parodia_score', 'Deporte_score',
       'Supervivencia_score', 'Aventura_score', 'Oeste_score',
       'Superpoderes_score', 'Ecchi_score', 'Tragedia_score', 'Fantasia_score',
       'Gore_score', 'Boys Love_score', 'Reencarnación_score',
       'Sobrenatural_score', 'Vida Escolar_score', 'Historia_score',
       'Romance_score', 'Ciencia 

# Preproceso

In [8]:
X = heavy_df.drop("score", axis=1)
y = heavy_df["score"]

In [9]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=2)

### Scaler

In [10]:
# Creo el scaler con los datos de train
scal = StandardScaler() # Declaro el scaler
scal.fit(X_train) # Lo "entreno". Calculo su media y std para cada feature
X_train_scal = scal.transform(X_train) # Aplico el scaler y sobreescribo los datos de train

# Aplico el mismo scaler con los datos de test
X_test_scal = scal.transform(X_test)


### PCA


In [11]:
# Aplicar PCA para reducir la dimensionalidad de los datos
pca = PCA(n_components=len(heavy_df.columns)-1)
# Ajustar y transformar los datos de entrenamiento utilizando PCA
X_train_pca = pca.fit_transform(X_train)
# Transformar los datos de prueba utilizando el modelo PCA ajustado previamente
X_test_pca = pca.transform(X_test)

In [12]:
# Obtener la proporción de varianza explicada por cada componente principal
pca.explained_variance_ratio_

array([8.93173668e-01, 8.22722831e-02, 1.50791122e-02, 8.44038962e-03,
       8.62368224e-04, 1.37313917e-04, 3.35779781e-05, 1.89174218e-07,
       1.41454164e-07, 9.52798455e-08, 9.29556238e-08, 8.88450134e-08,
       7.11694251e-08, 6.39104146e-08, 6.36044032e-08, 5.00914433e-08,
       4.75184813e-08, 3.66071975e-08, 3.05482094e-08, 2.76668056e-08,
       2.57723393e-08, 2.50899285e-08, 2.32635818e-08, 2.22550776e-08,
       1.97176872e-08, 1.80768223e-08, 1.77031774e-08, 1.62577271e-08,
       1.49094996e-08, 1.11453796e-08, 1.03924102e-08, 8.32043843e-09,
       8.09406541e-09, 7.03336654e-09, 5.73021480e-09, 4.92010350e-09,
       4.40508278e-09, 3.95026631e-09, 3.89684611e-09, 3.37473351e-09,
       3.19527398e-09, 3.02638857e-09, 2.80572509e-09, 2.73823700e-09,
       2.67704586e-09, 2.61457891e-09, 2.46273123e-09, 2.20982240e-09,
       2.10809814e-09, 1.94222758e-09, 1.79992192e-09, 1.58762127e-09,
       1.29152768e-09, 8.86043951e-10, 6.01275531e-10, 5.34585096e-10,
      

In [13]:
# Obtener la varianza explicada acumulada por cada componente principal
pca.explained_variance_ratio_.cumsum()

array([0.89317367, 0.97544595, 0.99052506, 0.99896545, 0.99982782,
       0.99996514, 0.99999871, 0.9999989 , 0.99999904, 0.99999914,
       0.99999923, 0.99999932, 0.99999939, 0.99999946, 0.99999952,
       0.99999957, 0.99999962, 0.99999965, 0.99999968, 0.99999971,
       0.99999974, 0.99999976, 0.99999979, 0.99999981, 0.99999983,
       0.99999985, 0.99999986, 0.99999988, 0.9999999 , 0.99999991,
       0.99999992, 0.99999993, 0.99999993, 0.99999994, 0.99999995,
       0.99999995, 0.99999996, 0.99999996, 0.99999996, 0.99999997,
       0.99999997, 0.99999997, 0.99999998, 0.99999998, 0.99999998,
       0.99999998, 0.99999999, 0.99999999, 0.99999999, 0.99999999,
       0.99999999, 1.        , 1.        , 1.        , 1.        ,
       1.        , 1.        , 1.        , 1.        , 1.        ])

# Polynomial + LinearRegression + PCA

- `poly_feats = PolynomialFeatures(degree=2)`: Se crea un objeto `PolynomialFeatures` con grado 2, lo que indica que se generarán características polinómicas hasta el segundo grado.

- `poly_feats.fit(X)`: Se ajusta el objeto `PolynomialFeatures` a los datos de entrada `X`. Esto calcula los coeficientes necesarios para generar las características polinómicas.

- `X_poly = poly_feats.transform(X)`: Se transforman los datos de entrada `X` utilizando el objeto `PolynomialFeatures` ajustado previamente. Esto agrega las características polinómicas al conjunto de datos original, generando así una representación de mayor dimensionalidad.

- `lin_reg = LinearRegression()`: Se crea un objeto `LinearRegression` para realizar una regresión lineal.

- `lin_reg.fit(X_train_pca, y_train)`: Se ajusta el modelo de regresión lineal utilizando los datos de entrenamiento con componentes principales `X_train_pca` y las etiquetas correspondientes `y_train`. Esto encuentra los coeficientes óptimos para la línea de regresión.


In [14]:
# Crear un objeto PolynomialFeatures para generar características polinómicas de grado 2
poly_feats = PolynomialFeatures(degree=2)

# Ajustar PolynomialFeatures a los datos de entrada X
poly_feats.fit(X)

# Transformar los datos de entrada X para incluir características polinómicas
X_poly = poly_feats.transform(X)

# Crear un objeto LinearRegression para realizar una regresión lineal
lin_reg = LinearRegression()

# Ajustar el modelo de regresión lineal utilizando los datos de entrenamiento con componentes principales
lin_reg.fit(X_train_pca, y_train)


LinearRegression()

In [15]:
parameter["plr"] = {"poli__degree": [2]}

In [16]:
predictions = lin_reg.predict(X_test_pca)

In [17]:
MAE = mean_absolute_error(y_test, predictions)
MAPE = mean_absolute_percentage_error(y_test, predictions)
MSE = mean_squared_error(y_test, predictions)
RMSE = np.sqrt(mean_squared_error(y_test, predictions))
R2_Train = lin_reg.score(X_train_pca, y_train)
R2_Test = lin_reg.score(X_test_pca, y_test)

In [18]:
score["plr"] = {"MAE": float(MAE), "MAPE": float(MAPE), "MSE": float(MSE), 
                "RMSE": float(RMSE), "R2_Train": float(R2_Train), "R2_Test": float(R2_Test)}

In [19]:
print("MAE:", MAE)
print("MAPE:", MAPE)
print("MSE:", MSE)
print("RMSE:", RMSE)
print("R2_Train:", R2_Train)
print("R2_Test:", R2_Test)

MAE: 1.6264203088426696
MAPE: 3241180710587130.5
MSE: 4.441959060468691
RMSE: 2.1075955637808432
R2_Train: 0.7037222469074746
R2_Test: 0.7067121408724921


# RandomForestRegressor

- `model = RandomForestRegressor(random_state=10)`: Se crea un objeto `RandomForestRegressor` con un estado aleatorio de 10. Este modelo se utiliza para realizar una regresión utilizando un conjunto de árboles de decisión.

- `parameters = {...}`: Se definen los parámetros para la búsqueda de hiperparámetros. En este caso, se especifican diferentes valores para los hiperparámetros como el número de estimadores (`n_estimators`), la profundidad máxima del árbol (`max_depth`), el número mínimo de muestras en las hojas del árbol (`min_samples_leaf`) y el número máximo de características a considerar en cada división (`max_features`).

- `dtr_gs = GridSearchCV(model, parameters, cv=5, scoring="neg_mean_absolute_error", n_jobs=-1)`: Se crea un objeto `GridSearchCV` para realizar una búsqueda exhaustiva de los mejores hiperparámetros para el modelo de regresión utilizando validación cruzada. Se especifica el modelo (`model`), los parámetros a buscar (`parameters`), el número de divisiones de la validación cruzada (`cv=5`), la métrica de evaluación (`scoring="neg_mean_absolute_error"`) y el número de trabajos en paralelo (`n_jobs=-1`) para acelerar el proceso de búsqueda.


In [20]:
# Crear un objeto RandomForestRegressor con un estado aleatorio de 10
model = RandomForestRegressor(random_state=10)

# Definir los parámetros para la búsqueda de hiperparámetros
parameters = {"n_estimators":[150,200],
            "max_depth":[20,21,22],
            "min_samples_leaf": [1,2,3],
            "max_features": [58,59,60]
}

# Crear un objeto GridSearchCV para realizar una búsqueda exhaustiva de hiperparámetros
dtr_gs = GridSearchCV(model, parameters, cv=5, scoring="neg_mean_absolute_error", n_jobs=-1)


In [21]:
dtr_gs.fit(X_train, y_train)

GridSearchCV(cv=5, estimator=RandomForestRegressor(random_state=10), n_jobs=-1,
             param_grid={'max_depth': [20, 21, 22],
                         'max_features': [58, 59, 60],
                         'min_samples_leaf': [1, 2, 3],
                         'n_estimators': [150, 200]},
             scoring='neg_mean_absolute_error')

In [22]:
best_params = {}
[best_params.update({f"regressor__{key}": [value]}) for key, value in dtr_gs.best_params_.items()]
parameter["rfr"] = best_params

In [23]:
feature_importances = dtr_gs.best_estimator_.feature_importances_
feature_importances_dict = dict(zip(X.columns, feature_importances))
feature_importances_sort = sorted(feature_importances_dict.items(), key=lambda x:x[1], reverse=True)

In [24]:
feature_importances_sort

[('tdg', 0.6250947915871793),
 ('pending', 0.0907002346519572),
 ('type_score', 0.0450274136510725),
 ('read', 0.03166637440404028),
 ('favorite', 0.026034749353240554),
 ('chapters_count', 0.022339008995303996),
 ('Vida Escolar_score', 0.019962191786438262),
 ('book_status_score', 0.01941602598877113),
 ('have', 0.014885559061074725),
 ('following', 0.014839904821373786),
 ('demography_score', 0.014217291892502093),
 ('abandoned', 0.010971386893193293),
 ('Acción_score', 0.006400333274423379),
 ('Aventura_score', 0.003399567040925213),
 ('Demonios_score', 0.003262877534876931),
 ('Sobrenatural_score', 0.003121806256740565),
 ('Fantasia_score', 0.0026797664029933935),
 ('Recuentos de la vida_score', 0.0023816256122150417),
 ('Boys Love_score', 0.0023763107528578563),
 ('Crimen_score', 0.0022716657157703835),
 ('Deporte_score', 0.002262467994271685),
 ('Romance_score', 0.0021328929363714014),
 ('Apocalíptico_score', 0.0021247150239417753),
 ('Comedia_score', 0.002055856124020214),
 ('Su

In [25]:
predictions = dtr_gs.predict(X_test)

In [26]:
MAE = mean_absolute_error(y_test, predictions)
MAPE = mean_absolute_percentage_error(y_test, predictions)
MSE = mean_squared_error(y_test, predictions)
RMSE = np.sqrt(mean_squared_error(y_test, predictions))
R2_Train = dtr_gs.best_estimator_.score(X_train, y_train)
R2_Test = dtr_gs.best_estimator_.score(X_test, y_test)

In [27]:
score["rfr"] = {"MAE": float(MAE), "MAPE": float(MAPE), "MSE": float(MSE), "RMSE": float(RMSE),
                "R2_Train": float(R2_Train), "R2_Test": float(R2_Test)}

In [28]:
print("MAE:", MAE)
print("MAPE:", MAPE)
print("MSE:", MSE)
print("RMSE:", RMSE)
print("R2_Train:", R2_Train)
print("R2_Test:", R2_Test)

MAE: 0.751342174053227
MAPE: 1314926994145115.2
MSE: 1.9080463851664147
RMSE: 1.3813205222418201
R2_Train: 0.9797271006145702
R2_Test: 0.8740180105661777


# ElasticNet

- `model = ElasticNet()`: Se crea un objeto `ElasticNet`. Este modelo combina la regularización L1 y L2 para realizar una regresión lineal con términos de penalización tanto en los coeficientes como en su magnitud.

- `parameters = {...}`: Se definen los parámetros para la búsqueda de hiperparámetros. En este caso, se especifican diferentes valores para los hiperparámetros `alpha`, que controla la fuerza de la regularización, `l1_ratio`, que determina la mezcla entre la regularización L1 y L2, y `max_iter`, que indica el número máximo de iteraciones del algoritmo de optimización.

- `dtr_gs = GridSearchCV(model, parameters, cv=5, scoring="neg_mean_absolute_error", n_jobs=-1)`: Se crea un objeto `GridSearchCV` para realizar una búsqueda exhaustiva de los mejores hiperparámetros para el modelo ElasticNet utilizando validación cruzada. Se especifica el modelo (`model`), los parámetros a buscar (`parameters`), el número de divisiones de la validación cruzada (`cv=5`), la métrica de evaluación (`scoring="neg_mean_absolute_error"`) y el número de trabajos en paralelo (`n_jobs=-1`) para acelerar el proceso de búsqueda.


In [29]:
# Crear un objeto ElasticNet
model = ElasticNet()

# Definir los parámetros para la búsqueda de hiperparámetros
parameters = {
    "alpha": [0.1, 0.5, 1.0],
    "l1_ratio": [0.3, 0.5, 0.7, 0.9],
    "max_iter": [1000, 2000]
}

# Crear un objeto GridSearchCV para realizar una búsqueda exhaustiva de hiperparámetros
dtr_gs = GridSearchCV(model, parameters, cv=5, scoring="neg_mean_absolute_error", n_jobs=-1)


In [30]:
dtr_gs.fit(X_train, y_train)

GridSearchCV(cv=5, estimator=ElasticNet(), n_jobs=-1,
             param_grid={'alpha': [0.1, 0.5, 1.0],
                         'l1_ratio': [0.3, 0.5, 0.7, 0.9],
                         'max_iter': [1000, 2000]},
             scoring='neg_mean_absolute_error')

In [31]:
best_params = {}
[best_params.update({f"regressor__{key}": [value]}) for key, value in dtr_gs.best_params_.items()]
parameter["en"] = best_params

In [32]:
predictions = dtr_gs.predict(X_test)

In [33]:
MAE = mean_absolute_error(y_test, predictions)
MAPE = mean_absolute_percentage_error(y_test, predictions)
MSE = mean_squared_error(y_test, predictions)
RMSE = np.sqrt(mean_squared_error(y_test, predictions))
R2_Train = dtr_gs.best_estimator_.score(X_train, y_train)
R2_Test = dtr_gs.best_estimator_.score(X_test, y_test)

In [34]:
score["en"] = {"MAE": float(MAE), "MAPE": float(MAPE), "MSE": float(MSE), "RMSE": float(RMSE), 
                "R2_Train": float(R2_Train), "R2_Test": float(R2_Test)}

In [35]:
print("MAE:", MAE)
print("MAPE:", MAPE)
print("MSE:", MSE)
print("RMSE:", RMSE)
print("R2_Train:", R2_Train)
print("R2_Test:", R2_Test)

MAE: 1.9736310535663755
MAPE: 4220003495667564.5
MSE: 5.984538432553491
RMSE: 2.4463316276730533
R2_Train: 0.602765646466361
R2_Test: 0.6048607290485233


# Support Vector Regression (SVR)

- `model = SVR()`: Se crea un objeto SVR (Support Vector Regression). Este modelo se utiliza para realizar una regresión utilizando la técnica de Support Vector Machines (SVM).

- `parameters = {...}`: Se definen los parámetros para la búsqueda de hiperparámetros. En este caso, se especifican diferentes valores para los hiperparámetros `kernel`, que determina el tipo de kernel a utilizar (`rbf` o `poly`), `C`, que controla la penalización por error de entrenamiento y la complejidad del modelo, y `epsilon`, que establece el margen de error permitido en la regresión.

- `dtr_gs = GridSearchCV(model, parameters, cv=5, scoring="neg_mean_absolute_error", n_jobs=-1)`: Se crea un objeto `GridSearchCV` para realizar una búsqueda exhaustiva de los mejores hiperparámetros para el modelo SVR utilizando validación cruzada. Se especifica el modelo (`model`), los parámetros a buscar (`parameters`), el número de divisiones de la validación cruzada (`cv=5`), la métrica de evaluación (`scoring="neg_mean_absolute_error"`) y el número de trabajos en paralelo (`n_jobs=-1`) para acelerar el proceso de búsqueda.


In [36]:
# Crear un objeto SVR
model = SVR()

# Definir los parámetros para la búsqueda de hiperparámetros
parameters = {
    "kernel": ["rbf", "poly"],
    "C": [0.5, 1, 2],
    "epsilon": [0.01, 0.1, 0.3],
}

# Crear un objeto GridSearchCV para realizar una búsqueda exhaustiva de hiperparámetros
dtr_gs = GridSearchCV(model, parameters, cv=5, scoring="neg_mean_absolute_error", n_jobs=-1)


In [37]:
dtr_gs.fit(X_train_scal, y_train)

GridSearchCV(cv=5, estimator=SVR(), n_jobs=-1,
             param_grid={'C': [0.5, 1, 2], 'epsilon': [0.01, 0.1, 0.3],
                         'kernel': ['rbf', 'poly']},
             scoring='neg_mean_absolute_error')

In [38]:
best_params = {}
[best_params.update({f"regressor__{key}": [value]}) for key, value in dtr_gs.best_params_.items()]
parameter["svr"] = best_params

In [39]:
dtr_gs.best_params_

{'C': 2, 'epsilon': 0.01, 'kernel': 'rbf'}

In [40]:
predictions = dtr_gs.predict(X_test_scal)

In [41]:
MAE = mean_absolute_error(y_test, predictions)
MAPE = mean_absolute_percentage_error(y_test, predictions)
MSE = mean_squared_error(y_test, predictions)
RMSE = np.sqrt(mean_squared_error(y_test, predictions))
R2_Train = dtr_gs.best_estimator_.score(X_train_scal, y_train)
R2_Test = dtr_gs.best_estimator_.score(X_test_scal, y_test)

In [42]:
score["svr"] = {"MAE": float(MAE), "MAPE": float(MAPE), "MSE": float(MSE), "RMSE": float(RMSE),
                "R2_Train": float(R2_Train), "R2_Test": float(R2_Test)}

In [43]:
print("MAE:", MAE)
print("MAPE:", MAPE)
print("MSE:", MSE)
print("RMSE:", RMSE)
print("R2_Train:", R2_Train)
print("R2_Test:", R2_Test)

MAE: 1.2523361553789056
MAPE: 3449866583858207.0
MSE: 4.464253488724695
RMSE: 2.1128780108479273
R2_Train: 0.7632441876465479
R2_Test: 0.7052401135429595


# AdaBoostRegressor

- `model = AdaBoostRegressor(random_state=10)`: Se crea un objeto AdaBoostRegressor con un estado aleatorio de 10. Este modelo se utiliza para realizar una regresión utilizando el algoritmo de Boosting con la técnica de AdaBoost.

- `parameters = {...}`: Se definen los parámetros para la búsqueda de hiperparámetros. En este caso, se especifican diferentes valores para los hiperparámetros `n_estimators`, que indica el número de estimadores en el ensamblaje, y `loss`, que determina la función de pérdida a utilizar durante el entrenamiento.

- `dtr_gs = GridSearchCV(model, parameters, cv=5, scoring="neg_mean_absolute_error", n_jobs=-1)`: Se crea un objeto GridSearchCV para realizar una búsqueda exhaustiva de los mejores hiperparámetros para el modelo AdaBoostRegressor utilizando validación cruzada. Se especifica el modelo (`model`), los parámetros a buscar (`parameters`), el número de divisiones de la validación cruzada (`cv=5`), la métrica de evaluación (`scoring="neg_mean_absolute_error"`) y el número de trabajos en paralelo (`n_jobs=-1`) para acelerar el proceso de búsqueda.


In [44]:
# Crear un objeto AdaBoostRegressor
model = AdaBoostRegressor(random_state=10)

# Definir los parámetros para la búsqueda de hiperparámetros
parameters = {
    "n_estimators": [100, 150, 200],
    "loss": ["linear", "square", "exponential"],
}

# Crear un objeto GridSearchCV para realizar una búsqueda exhaustiva de hiperparámetros
dtr_gs = GridSearchCV(model, parameters, cv=5, scoring="neg_mean_absolute_error", n_jobs=-1)


In [45]:
dtr_gs.fit(X_train, y_train)

GridSearchCV(cv=5, estimator=AdaBoostRegressor(random_state=10), n_jobs=-1,
             param_grid={'loss': ['linear', 'square', 'exponential'],
                         'n_estimators': [100, 150, 200]},
             scoring='neg_mean_absolute_error')

In [46]:
best_params = {}
[best_params.update({f"regressor__{key}": [value]}) for key, value in dtr_gs.best_params_.items()]
parameter["ada"] = best_params

In [47]:
dtr_gs.best_params_

{'loss': 'linear', 'n_estimators': 100}

In [48]:
predictions = dtr_gs.predict(X_test)

In [49]:
MAE = mean_absolute_error(y_test, predictions)
MAPE = mean_absolute_percentage_error(y_test, predictions)
MSE = mean_squared_error(y_test, predictions)
RMSE = np.sqrt(mean_squared_error(y_test, predictions))
R2_Train = dtr_gs.best_estimator_.score(X_train, y_train)
R2_Test = dtr_gs.best_estimator_.score(X_test, y_test)

In [50]:
score["abr"] = {"MAE": float(MAE), "MAPE": float(MAPE), "MSE": float(MSE), "RMSE": float(RMSE),
                "R2_Train": float(R2_Train), "R2_Test": float(R2_Test)}

In [51]:
print("MAE:", MAE)
print("MAPE:", MAPE)
print("MSE:", MSE)
print("RMSE:", RMSE)
print("R2_Train:", R2_Train)
print("R2_Test:", R2_Test)

MAE: 2.2864885026138806
MAPE: 4299650956092424.5
MSE: 7.4031239197150605
RMSE: 2.720868229024526
R2_Train: 0.5099155703618581
R2_Test: 0.5111962231728051


# GradientBoostingRegressor

Importamos la clase `GradientBoostingRegressor` del módulo `ensemble` de la biblioteca sklearn. Esto nos permite utilizar el algoritmo de Boosting con gradientes para realizar una regresión.

- `max_depth`: La profundidad máxima de los árboles de regresión utilizados en el ensamble. En este caso, se establece en 2.
- `n_estimators`: El número de árboles en el ensamble. En este caso, se establece en 3.
- `learning_rate`: La tasa de aprendizaje que controla la contribución de cada árbol en el ensamble. En este caso, se establece en 1.0, lo que significa que cada árbol contribuye completamente al ensamble.
- `random_state`: El estado aleatorio para reproducibilidad. En este caso, se establece en 10.


In [52]:
from sklearn.ensemble import GradientBoostingRegressor

# Crear un objeto GradientBoostingRegressor
gbrt = GradientBoostingRegressor(max_depth=2,
                                 n_estimators=3,
                                 learning_rate=1.0,
                                 random_state=10)

# Ajustar el modelo GradientBoostingRegressor a los datos de entrenamiento
gbrt.fit(X_train, y_train)


GradientBoostingRegressor(learning_rate=1.0, max_depth=2, n_estimators=3,
                          random_state=10)

In [53]:
predictions = gbrt.predict(X_test)

In [54]:
MAE = mean_absolute_error(y_test, predictions)
MAPE = mean_absolute_percentage_error(y_test, predictions)
MSE = mean_squared_error(y_test, predictions)
RMSE = np.sqrt(mean_squared_error(y_test, predictions))
R2_Train = dtr_gs.best_estimator_.score(X_train, y_train)
R2_Test = dtr_gs.best_estimator_.score(X_test, y_test)

In [55]:
score["gbr"] = {"MAE": float(MAE), "MAPE": float(MAPE), "MSE": float(MSE), "RMSE": float(RMSE),
                "R2_Train": float(R2_Train), "R2_Test": float(R2_Test)}

In [56]:
print("MAE:", MAE)
print("MAPE:", MAPE)
print("MSE:", MSE)
print("RMSE:", RMSE)
print("R2_Train:", R2_Train)
print("R2_Test:", R2_Test)

MAE: 1.3369687018314675
MAPE: 2305030290860392.5
MSE: 4.4187174660129
RMSE: 2.102074562429435
R2_Train: 0.5099155703618581
R2_Test: 0.5111962231728051


# Recurrent Neural Networks

**Model Creation:**

- `model`: Se crea un modelo secuencial de Keras utilizando la clase `Sequential`. Este modelo consiste en una serie de capas apiladas una encima de la otra.
- `keras.layers.Dense`: Se añaden capas densas (totalmente conectadas) al modelo. Cada capa tiene un número específico de neuronas y una función de activación. La última capa tiene 1 neurona para la regresión.

**Model Compilation:**

- `model.compile`: Se compila el modelo especificando la función de pérdida, el optimizador y las métricas que se utilizarán durante el entrenamiento.

**Callbacks:**

- `checkpoint_cb`: Se crea un callback `ModelCheckpoint` para guardar el modelo en un archivo durante el entrenamiento.
- `early_stopping_cb`: Se crea un callback `EarlyStopping` que detendrá el entrenamiento si no se observa una mejora en la métrica de pérdida en un número determinado de épocas.

**Model Training:**

- `model.fit`: Se entrena el modelo utilizando los datos de entrenamiento `X_train_scal` y las etiquetas correspondientes `y_train`. Se especifica el número de épocas, la validación cruzada (usando una fracción del conjunto de entrenamiento como validación) y los callbacks a utilizar durante el entrenamiento.


In [57]:
model = keras.Sequential([
    keras.layers.Dense(120, activation='relu', input_shape=X_train_scal.shape[1:]),
    keras.layers.Dense(60, activation='relu'),
    keras.layers.Dense(30, activation='relu'),
    keras.layers.Dense(1, activation='relu')
])

model.compile(loss='mae', optimizer=keras.optimizers.Adam(), 
                metrics=[keras.metrics.MeanAbsolutePercentageError(name="MAPE"),
                        keras.metrics.MeanSquaredError(name='MSE'),
                        keras.metrics.RootMeanSquaredError(name="RMSE")
                        ])

checkpoint_cb = keras.callbacks.ModelCheckpoint("../models/heavy/callback_model.h5")
early_stopping_cb = keras.callbacks.EarlyStopping(monitor='val_loss', patience=10)

history = model.fit(X_train_scal, y_train, epochs=1000, validation_split=0.2,
                   callbacks = [early_stopping_cb, checkpoint_cb])

Epoch 1/1000
381/381 [==============================] - 2s 2ms/step - loss: 1.9317 - MAPE: 510327456.0000 - MSE: 9.5416 - RMSE: 3.0889 - val_loss: 0.8850 - val_MAPE: 165536448.0000 - val_MSE: 3.0381 - val_RMSE: 1.7430
Epoch 2/1000
381/381 [==============================] - 1s 2ms/step - loss: 0.7400 - MAPE: 136627168.0000 - MSE: 2.4243 - RMSE: 1.5570 - val_loss: 0.6835 - val_MAPE: 106362680.0000 - val_MSE: 2.4086 - val_RMSE: 1.5520
Epoch 3/1000
381/381 [==============================] - 1s 2ms/step - loss: 0.6008 - MAPE: 80285152.0000 - MSE: 2.0994 - RMSE: 1.4489 - val_loss: 0.6061 - val_MAPE: 73409104.0000 - val_MSE: 2.2465 - val_RMSE: 1.4988
Epoch 4/1000
381/381 [==============================] - 1s 2ms/step - loss: 0.5254 - MAPE: 53034480.0000 - MSE: 1.8820 - RMSE: 1.3719 - val_loss: 0.5744 - val_MAPE: 69008600.0000 - val_MSE: 2.2197 - val_RMSE: 1.4899
Epoch 5/1000
381/381 [==============================] - 1s 2ms/step - loss: 0.4797 - MAPE: 44766456.0000 - MSE: 1.7353 - RMSE: 1.317

In [58]:
rnn_score = model.evaluate(X_test_scal, y_test)


119/119 [==============================] - 0s 1ms/step - loss: 0.4382 - MAPE: 45000216.0000 - MSE: 2.1034 - RMSE: 1.4503


In [59]:
MAE = rnn_score[0]
MAPE = rnn_score[1]
MSE = rnn_score[2]
RMSE = rnn_score[3]

In [60]:
score["rnn"] = {"MAE": float(MAE), "MAPE": float(MAPE), "MSE": float(MSE), "RMSE": float(RMSE)}

In [61]:
print("MAE:", MAE)
print("MAPE:", MAPE)
print("MSE:", MSE)
print("RMSE:", RMSE)

MAE: 0.43818891048431396
MAPE: 45000216.0
MSE: 2.1033761501312256
RMSE: 1.4503021240234375


In [62]:
model.save("../models/heavy/keras_model.h5")

# Summary

In [63]:
score

{'plr': {'MAE': 1.6264203088426696,
  'MAPE': 3241180710587130.5,
  'MSE': 4.441959060468691,
  'RMSE': 2.1075955637808432,
  'R2_Train': 0.7037222469074746,
  'R2_Test': 0.7067121408724921},
 'rfr': {'MAE': 0.751342174053227,
  'MAPE': 1314926994145115.2,
  'MSE': 1.9080463851664147,
  'RMSE': 1.3813205222418201,
  'R2_Train': 0.9797271006145702,
  'R2_Test': 0.8740180105661777},
 'en': {'MAE': 1.9736310535663755,
  'MAPE': 4220003495667564.5,
  'MSE': 5.984538432553491,
  'RMSE': 2.4463316276730533,
  'R2_Train': 0.602765646466361,
  'R2_Test': 0.6048607290485233},
 'svr': {'MAE': 1.2523361553789056,
  'MAPE': 3449866583858207.0,
  'MSE': 4.464253488724695,
  'RMSE': 2.1128780108479273,
  'R2_Train': 0.7632441876465479,
  'R2_Test': 0.7052401135429595},
 'abr': {'MAE': 2.2864885026138806,
  'MAPE': 4299650956092424.5,
  'MSE': 7.4031239197150605,
  'RMSE': 2.720868229024526,
  'R2_Train': 0.5099155703618581,
  'R2_Test': 0.5111962231728051},
 'gbr': {'MAE': 1.3369687018314675,
  'MAP

In [64]:
parameter

{'plr': {'poli__degree': [2]},
 'rfr': {'regressor__max_depth': [21],
  'regressor__max_features': [60],
  'regressor__min_samples_leaf': [1],
  'regressor__n_estimators': [150]},
 'en': {'regressor__alpha': [0.1],
  'regressor__l1_ratio': [0.9],
  'regressor__max_iter': [1000]},
 'svr': {'regressor__C': [2],
  'regressor__epsilon': [0.01],
  'regressor__kernel': ['rbf']},
 'ada': {'regressor__loss': ['linear'], 'regressor__n_estimators': [100]}}

In [65]:
write_yaml("../models/heavy/model_config.yaml", parameter)

YAML file write: OK


True

In [66]:
write_yaml("../models/heavy/score.yaml", score)

YAML file write: OK


True

In [67]:
bee()